## Run the blocks below only for the first time
### Install dependencies

In [1]:
%%bash
sh env.sh

Couldn't find program: 'bash'


### Download pretrained models (may take a minute)

In [2]:
import yaml
import sys
import requests
import os
import re
import tarfile
import shutil
# for zip file
import zipfile


# url = "https://onedrive.live.com/?cid=5435770760f02d2f&id=5435770760F02D2F%211152&authkey=!AD6jcLtyet8Y3o4"
    
# r = requests.get(url, allow_redirects=True)

# open('cpvton-plus-checkpoints.zip', 'wb').write(r.content)

def download(url, filename, cookies=None):
    with open(filename, 'wb') as f:
        response = requests.get(url, stream=True, cookies=cookies)
        total = response.headers.get('content-length')

        if total is None:
            f.write(response.content)
        else:
            downloaded = 0
            total = int(total)
            for data in response.iter_content(chunk_size=max(int(total/1000), 1024*1024)):
                downloaded += len(data)
                f.write(data)
                completed = int(50*downloaded/total)
                sys.stdout.write('\r[{}{}]'.format(
                    '█' * completed, '.' * (50-completed)))
                sys.stdout.flush()
    sys.stdout.write('\n')
drive_request = requests.get(
    'https://drive.google.com/uc?export=download&confirm=CONFIRM&id=199dwgFGlumJ95MAGNbLnYtbVKA2w6kdc')
confirm_page = drive_request.text
confirmation_code = re.findall('confirm=(.{4})', confirm_page)[0]

print('[*] Downloading data...')
download('https://drive.google.com/uc?export=download&confirm=CONFIRM&id=199dwgFGlumJ95MAGNbLnYtbVKA2w6kdc'.replace(
    'CONFIRM', confirmation_code), 'cpvton-plus-checkpoints.zip', cookies=drive_request.cookies)
print('[^] Download complete')

[*] Downloading data...

[^] Download complete


### Excuate the block below once you see a 'cpvton-plus-checkpoints.zip' in the storage directory
#### Extract the checkpoint to 'storage/checkpoints-cpvton-plus' directory and delete the zip file

In [3]:
if not os.path.exists('checkpoints-cpvton-plus'):
    os.makedirs('checkpoints-cpvton-plus')
checkpoint = zipfile.ZipFile("cpvton-plus-checkpoints.zip")
checkpoint.extractall(path='checkpoints-cpvton-plus')
shutil.move('checkpoints-cpvton-plus/checkpoints/GMM/', 'checkpoints-cpvton-plus/GMM/')
shutil.move('checkpoints-cpvton-plus/checkpoints/TOM/', 'checkpoints-cpvton-plus/TOM/')
os.rmdir('checkpoints-cpvton-plus/checkpoints')
os.remove('cpvton-plus-checkpoints.zip')

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'cpvton-plus-checkpoints.zip'

### Download the whole VITON datasets and relocate the data (may take 20 minutes or so)

In [4]:
if not os.path.exists('data'):
    os.makedirs('data')

In [5]:

drive_request = requests.get(
    'https://drive.google.com/uc?export=download&confirm=CONFIRM&id=1MxCUvKxejnwWnoZ-KoCyMCXo3TLhRuTo')
confirm_page = drive_request.text
confirmation_code = re.findall('confirm=(.{4})', confirm_page)[0]

print('[*] Downloading data...')
download('https://drive.google.com/uc?export=download&confirm=CONFIRM&id=1MxCUvKxejnwWnoZ-KoCyMCXo3TLhRuTo'.replace(
    'CONFIRM', confirmation_code), 'data/viton_resize.tar.gz', cookies=drive_request.cookies)

tarfile.open("data/viton_resize.tar.gz").extractall(path='data/')

shutil.move('data/viton_resize/test/', 'data/test/')
shutil.move('data/viton_resize/train/', 'data/train/')

os.rmdir('data/viton_resize/')
os.remove('data/viton_resize.tar.gz')

[*] Downloading data...



## Run the blocks above only for the first time
### Run blocks below for test stage
#### Change directory to root 
##### Run blocks below once you see train and test folders in 'storage/data' directory

In [6]:
import os
os.chdir('../notebooks')
if (os. getcwd() == '/notebooks'):
    print('Now working in the right location')
else:
    print('ERROE! Current directory is:', os. getcwd())
        

FileNotFoundError: [WinError 2] The system cannot find the file specified: '../notebooks'

### Build GMM results to wrap the clothes based on test models' poses (I changed the python scripts directly so below commends run by bash)
### Each lines may take a long time to run
#### You can check the results after the procedure finished

### Preprocessing Stage 
#### Generate stage one sagementation results with neck features

In [7]:
%%bash
python dataset_neck_skin_correction.py

Couldn't find program: 'bash'


In [ ]:
%%bash
python body_binary_masking.py

### GMM results

In [8]:
%%bash
python test.py --name GMM --dataroot storage/data --stage GMM --workers 4 --datamode test --data_list test_pairs.txt --checkpoint storage/checkpoints-cpvton-plus/GMM/gmm_final.pth


Couldn't find program: 'bash'


In [9]:
%%bash
cp -r result/GMM/test/warp-cloth storage/data/test
cp -r result/GMM/test/warp-mask storage/data/test



Couldn't find program: 'bash'


In [10]:
%%bash
python test.py --name TOM --dataroot storage/data --stage TOM --workers 4 --datamode test --data_list test_pairs.txt --checkpoint storage/checkpoints-cpvton-plus/TOM/tom_final.pth

Couldn't find program: 'bash'


In [11]:
%%bash
mv -r result storage/result

Couldn't find program: 'bash'


 ### The results for cp-vton-plus now can be checked in 'storage/result/TOM/test/try-on'

### Training Stage

In [ ]:
%%bash
python train.py --name GMM --dataroot storage/data --stage GMM --workers 4 --checkpoint_dir storage/checkpoints-cpvton-plus/GMM/ --save_count 5000 --shuffle